In [1]:
!pip install tensorflow
import numpy as np
import os
import tensorflow as tf



You are using pip version 19.0.3, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
#model_name = 'distilbert-base-nli-stsb-mean-tokens'
model_name ='paraphrase-distilroberta-base-v1'
npzfile = np.load('train_val_test_'+model_name+'.npz',allow_pickle=True)
npzfile.files

['train_features',
 'train_labels',
 'val_features',
 'val_labels',
 'test_features',
 'test_labels',
 'label_maps',
 'allow_pickle']

In [3]:
train_features = npzfile['train_features']
train_labels = npzfile['train_labels']
val_features = npzfile['val_features']
val_labels = npzfile['val_labels']
label_map, label_map_inv = npzfile['label_maps']
D=train_features.shape[1]
C= len(label_map_inv.keys())

In [4]:
#one hot
train_labels_oh = tf.one_hot(train_labels, depth = C)
val_labels_oh = tf.one_hot(val_labels, depth = C)
#test_labels_oh = tf.one_hot(test_labels, depth = C)

In [5]:
train_labels

array([32, 32, 32, ..., 34, 34, 34])

In [6]:
len(label_map_inv.keys())

150

In [16]:
dr_rate=0.2

input_layer =  tf.keras.layers.Input(shape=(D,))
out = tf.keras.layers.Dense(1024, activation='relu') (input_layer)
out = tf.keras.layers.Dropout(dr_rate) (out)
out = tf.keras.layers.Dense(1024, activation='relu')(out)
out = tf.keras.layers.Dropout(dr_rate) (out)
out = tf.keras.layers.Dense(512, activation='relu')(out)
out = tf.keras.layers.Dropout(dr_rate)(out)
out = tf.keras.layers.Dense(256, activation='relu')(out)
out = tf.keras.layers.Dropout(dr_rate)(out)
out = tf.keras.layers.Dense(C) (out)

model = tf.keras.Model(input_layer, out)
    
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])




In [17]:

def save_model_json(model, path):

    print("Saving model  structure as json")

    with open(path, "w") as text_file:
        text_file.write(model.to_json())
    print("Done")


In [18]:
#Training of model
model_save_dir = 'savedmodel'
if(not os.path.exists(model_save_dir)):
    os.mkdir(model_save_dir)
model_filepath = os.path.join(model_save_dir,   'intent_clf.hd5') 
save_model_json(model, os.path.join(model_save_dir,   'intent_clf.json'))
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='auto', factor=0.5, patience=2, min_lr=1e-6)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_filepath, monitor='val_loss', verbose=1, save_best_only=True,
                                               save_weights_only=True, mode='auto', save_freq='epoch')


model.fit(train_features, train_labels_oh, validation_data = (val_features,val_labels_oh), 
          epochs=20, callbacks=[reduce_lr, model_checkpoint])

Saving model  structure as json
Done
Epoch 1/20
329/329 [==============================] - ETA: 0s - loss: 1.9494 - accuracy: 0.5414
Epoch 00001: val_loss improved from inf to 0.84857, saving model to savedmodel/intent_clf.hd5
329/329 [==============================] - 5s 16ms/step - loss: 1.9494 - accuracy: 0.5414 - val_loss: 0.8486 - val_accuracy: 0.7687
Epoch 2/20
325/329 [============================>.] - ETA: 0s - loss: 0.4552 - accuracy: 0.8692
Epoch 00002: val_loss improved from 0.84857 to 0.49249, saving model to savedmodel/intent_clf.hd5
329/329 [==============================] - 5s 16ms/step - loss: 0.4526 - accuracy: 0.8701 - val_loss: 0.4925 - val_accuracy: 0.8727
Epoch 3/20
327/329 [============================>.] - ETA: 0s - loss: 0.2458 - accuracy: 0.9293
Epoch 00003: val_loss improved from 0.49249 to 0.47019, saving model to savedmodel/intent_clf.hd5
329/329 [==============================] - 5s 15ms/step - loss: 0.2454 - accuracy: 0.9295 - val_loss: 0.4702 - val_accura